# Uninformed & Informed Search

2021.06.23


**Jeong - Jinha **






## Objectives


*   Read data from a file and Create a graph
*   Implement Uninformed & Informed searching strategies
*   Apply different searching strategies for the given problem
*   Analyze and Compare the searching strategies


## Description

This project is focused on **python file reading, graph creation** and implementation of **search algorithms**. 

*   Coronavirus is non-discriminatory, in the sense that it can spread from one city to any other city. The only goal of the virus is to spread to all cities in Texas. Using Uniformed Search, find a possible way for the Coronavirus to spread.
*   To counter the effect of the virus, vaccine needs to be distributed to all cities. One city has more demand than supply, whereas one city has a shortage of vaccines. The goal is to find an **optimal** strategy to transport the vaccine (Informed Search) from the city with high supply (low demand) to the city with low supply (high demand).


#Reading Data Files & Creating a 2D Graph

There are 2 csv files:

*   **cities.csv** - This file contains a list of coordinates for selected cities in Texas, in the following format:
```
San Antonio,29.4685,-98.5254
```
The above line means that San Antonio is located at the latitude and longitude of 29.4685 N and 98.5254 W respectively. Note that the '-ve' sign denotes 'S' for latitude and 'W' for longitude.

*   **distances.csv** - This file contains straight-line distance values between two cities of Texas, if a path exists, in the following format:
```
San Antonio,New Braunfels,30.80876734
```
The above line denotes that there should be an edge between *San Antonio* and *New Braunfels* and the weight on that edge, i.e. the distance, is *30.80876734*.




In [ ]:
import os
import re
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
#from geopy.distance import vincenty
#from queue import PriorityQueue
!pip install haversine
from haversine import haversine



from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
basePath = "/content/drive/My Drive/Colab Notebooks/Artificial Intelligence/Data/"
cities=os.path.join(basePath,"cities.csv")
print(cities)
distances=os.path.join(basePath,"distances.csv")
print(distances)

# pngName = os.path.join(basePath, 'texas.png')
# print(png)

city = open(cities,'r')
cities = city.readlines()

distance = open(distances,'r')
distances = distance.readlines()


/content/drive/My Drive/Colab Notebooks/Artificial Intelligence/Data/cities.csv
/content/drive/My Drive/Colab Notebooks/Artificial Intelligence/Data/distances.csv


In [ ]:
# Load the graph data from the files

# Processing the data 
# cities.csv
def splitThemForCities (raw_clue):
    a = list ()
    b = list ()
    c = list ()
    for i in range (len (raw_clue)):
        temp = raw_clue[i].split (',')
        a.append (temp[0])
        b.append (float(temp[1]))
        c.append (float(temp[2]))
    return (a, b, c) 

# distances.csv
def splitThemForDis (raw_clue):
    a = list ()
    b = list ()
    c = list ()
    for i in range (len (raw_clue)):
        temp = raw_clue[i].split (',')
        a.append (temp[0])
        b.append (temp[1])
        c.append (float(temp[2]))
    return (a, b, c) 

(city_name, city_x, city_y) = splitThemForCities(cities)
map_info = { 'city': city_name, 'latitude': city_x, 'longitude': city_y}
map_info = pd.DataFrame(map_info)
#print(map_info)


# latitude and longitude of College Station
#CS_coord = (map_info.loc[6, 'latitude'] , map_info.loc[6, 'longitude'])



#weight = calVincenty(CS_coord, map_info)

#map_info['weight'] = weight


(loc_x,loc_y, distance) = splitThemForDis(distances)
dis_info = { 'from' : loc_x, 'to' : loc_y, 'dist' : distance  }
dis_info = pd.DataFrame(dis_info)


# trying to learn the total distances traveled when using BFS.
# total_dist = 0
# for i in dis_info.index:
#   total_dist += dis_info.loc[i, 'dist']
# print(total_dist) 
# 4073.6619521800003 <-

#nodes and edges
G = nx.Graph()
for i in map_info.index:
  G.add_node(map_info.loc[i, 'city'], pos = (map_info.loc[i, 'longitude'],map_info.loc[i, 'latitude']))
for i in dis_info.index:
  G.add_edge(dis_info.loc[i, 'from'], dis_info.loc[i, 'to'], length = dis_info.loc[i, 'dist'])



#with_labels를 True로 전환하면 각 노드마다 도시 이름 출력됨
plt.figure(1, figsize=(30,30)) # 그래프의 전체 틀
nx.draw(G, nx.get_node_attributes(G, 'pos'), with_labels=True, node_color= 'skyblue' ,node_size=100, font_size =24, linewidths=8)
nx.draw_networkx_edge_labels(G, nx.get_node_attributes(G, 'pos'), edge_labels = nx.get_edge_attributes(G,'length'), label_pos = 0.5, font_size=12)
plt.show()

In [ ]:
# Display a 2D graph of the given data.

# discraded
# image = plt.imread(pngName,0) #someone changed the extension of the image from jpeg to png
# plt.figure(2, figsize=(30,30))
# plt.imshow(image)
# nx.draw(G, nx.get_node_attributes(G, 'pos'), with_labels=True, node_color= 'skyblue' ,node_size=100, font_size =24, linewidths=8)
# nx.draw_networkx_edge_labels(G, nx.get_node_attributes(G, 'pos'), edge_labels = nx.get_edge_attributes(G,'length'), label_pos = 0.5, font_size=12)
# plt.show()

import folium

lat = map_info['latitude'].mean()
long = map_info['longitude'].mean()

#display a graph on the map
m = folium.Map([lat,long],zoom_start=6)
plt.figure(2, figsize=(12,12))
for i in map_info.index:
  sub_lat = map_info.loc[i, 'latitude']
  sub_long = map_info.loc[i, 'longitude']
  title = map_info.loc[i, 'city']
  folium.CircleMarker([sub_lat,sub_long],tooltip = title).add_to(m)
# if you put your mouse on the markers on the map below, the tooltips will pop up and show you the name of the cities
plt.show()
m

#Virus Spread - Uninformed Search Agent



In this senario, the first confirmed case of the virus was in **Three Rivers** and starts spreading from there. The virus does not discriminate and it needs to spread to all the cities of Texas. We will use the graph created in the previous section and create an *uninformed search* agent that will print the path how the virus will spread to all the provided Texas cities. 


In [ ]:
# Implement ANY uninformed search strategy for the spread of coronavirus from the starting city of 'Three Rivers'

visited = list()
dist_list = list()
def BFS(graph, node, dist_list, visited):
    q = [node]
    visited.append(node)
    while len(q) > 0:
        dq= q.pop(0)
        for k,v in graph[dq].items():
            if k not in visited:
                visited.append(k)
                q.append(k)
                dist_list.append(v)
    return  dist_list, visited

BFS(G, 'Three Rivers', dist_list, visited)

distance_traveled = 0
for i in range(len(dist_list)):
  dist_list[i] = list(dist_list[i].values())
  distance_traveled += float(dist_list[i][0])

print('The # of Cities Traveled: %d \n' % len(visited))
for i in range(len(visited)):
  print(visited[i])
print('\nTotal Distance Traveled: %f miles' % distance_traveled)




#Vaccine Transportation - Informed Search Agent

In this senario, the city of **San Antonio** has more supply of vaccine than the demand. The goal is to create an **optimal strategy** to transport the vaccine and make it available at the highly affected city of **College Station**, where there is a shortage of vaccines.

In [ ]:
# calculate the straight distances of cities from College Station 
## Aborted vincenty because it takes too much time to calculate
# def calVincenty(a, b):
#   h_x =vincenty(a, b).miles
#   return h_x

def cordinate(city):
  cord = map_info.loc[map_info['city'] == city]
  cord = cord.values.tolist()
  cord = cord[0][1:]
  return cord



def AStarSearch(graph, start, goal, visited=[]):
    target = cordinate(goal)
    openList = []
    closedList = []
    h = []
    q = [start]
    goal = [goal]
    visited.append(start)
    closedList.append(start)

    while (q != goal):
        dq = q.pop(0)
        #연결된 도시 리스트 San Antonio, Austin 등등
        for i in graph[dq]:
          if (i not in visited and i not in openList):
            openList.append(i)
        #각각의 도시들의 휴리스틱 목표지점과 출발지점으로 부터의 거리 더하기
        for i in range(len(openList)):
          h.append(haversine(cordinate(openList[i]), target) + haversine(cordinate(openList[i]), cord(dq)))

        # h중에서 가장 작은 녀석의 값을 이용해 index 넘버 찾기
        count = h.index(min(h))
        h = []
        #방문한 노드에 기록
        visited.append(openList[count])
        #큐에 시작 노드로 정하기
        q.append(openList[count])
        #가장 작은 값을 여기에 더하기
        closedList.append(openList[count])
        #더한 녀석은 없애기
        openList.remove(openList[count])
        

    return closedList


shortest = AStarSearch(G, 'San Antonio', 'College Station')




print('The shortest path: %d \n' % len(shortest))
for i in range(len(shortest)):
  print(shortest[i])
print()

#distance_csv must be a DataFrame
def calDistance(seq):
  total_distance = 0
  for x in dis_info.index:
    for y in range(len(seq)-1):
      for z in range(1 ,len(seq)):
        if (seq[y] == dis_info.loc[x, 'from'] and seq[z] == dis_info.loc[x, 'to']): 
          total_distance += dis_info.loc[x, 'dist']
  return total_distance

traveled = calDistance(shortest)
   
print('Total distance traveled:'+str(traveled)+' miles')


The shortest path: 5 

San Antonio
New Braunfels
San Marcos
Austin
College Station

Total distance traveled:186.77450141 miles


In [ ]:
# Implement an OPTIMAL informed search strategy for distributing the vaccine from 'Corpus Chriti' to 'Amarillo'
# print(G.nodes['College Station']['weight'])
# print(nx.shortest_path(G, source ='San Antonio', target = 'College Station'))
# from heapq import heappush, heappop
# path =list()
# def aStar(graph, start, end):
#   q = [start]
#   goal = [end]
#   openList =[]
#   openList.append[start]
#   closedtList =[]

# while openList:
#   currentNode = openList[0]
#   currentIdx = 0
#   for index, item in enumerate(openList):
#     currentNode = item
#     currentIdx = index
#   openList.pop(currentIdx)
#   closedList.append(currentNode)

#   if currentNode == goal:
#     path = []
#     current = currentNode
#     while current

# import pdb

# visited = list()
# dist_list= list()
# def RBFS(graph, start, goal, distance, visited):
#   pq = PriorityQueue()
#   # pdb.set_trace()
#   pq.put((0,graph.nodes[start]['weight']))
#   visited.append(start)
#   print(visited)
#   while pq.empty() == False:
#     #dq가 예제에서는 가장 작은 수를 쫓아갔듯이 여기서는 distance or weight을 쫓아야함.
#       dq = pq.get()[1]
#       if dq == goal:
#         break      
#       for k,v in graph[dq].items():        
#         if k not in visited:
#           visited.append(k)
#           pq.put(k)
#           dist_list.append(v)
#   return  dist_list, visited

# RBFS(G, "San Antonio", "College Station", dist_list, visited)